importing packages

In [1]:
import pandas as pd
import matplotlib.pyplot as plt
from carriageway import Carriageway
from impact_factor import impact
from load import ll_A, ll_70R, ll_70RT
from reaction import find_bm, find_sf

defining load

In [2]:
vehicles = [ll_A, ll_70R, ll_70RT]
classA_pair, class70R, class70RT = [list(i.loadpair) for i in vehicles]
loads = [classA_pair, class70R, class70RT]

make an array for maxBM, maxSF at different intervals

In [3]:
span = 50

maxBMs = []
maxSFs_plus = []
maxSFs_minus = []
sections = []
for i in range(len(loads)):
    maxBM = []
    maxSF_plus = []
    maxSF_minus = []
    for j in range(9):
        at = span / 8 * j
        if at not in sections:
            sections.append(at)
        first_wheel_at = 0
        step = 0.1

        BM = find_bm(span, 0, at)
        SF_plus = find_sf(span, 0, at)
        SF_minus = find_sf(span, 0, at)
        # print(list(loads[i]))
        for k in range(int((span + loads[i][-1][0]) / step) + 2):  # '+2' to make sure the load moves all the way to
            # end until it has no effect
            bm = 0
            sf = 0
            for this in loads[i]:
                a, load = this
                pos = -a + first_wheel_at
                bm = bm + find_bm(span, pos, at) * load
                sf = sf + find_sf(span, pos, at) * load
            first_wheel_at += step
            BM = bm if bm > BM else BM
            SF_plus = sf if sf > SF_plus else SF_plus
            SF_minus = sf if sf < SF_minus else SF_minus
        maxBM.append(round(BM, 3))
        maxSF_plus.append(round(SF_plus, 3))
        maxSF_minus.append(round(SF_minus, 3))
    maxBMs.append(maxBM)
    maxSFs_plus.append(maxSF_plus)
    maxSFs_minus.append(maxSF_minus)

make a dataframe

In [4]:
A = ['MaxBM', 'MaxSF+', 'MaxSF-']
B = ['ClassA', 'Class70RW', 'Class70RT']

# A = ['ClassA', 'Class70RW', 'Class70RT']
# B = ['MaxBM', 'MaxSF+', 'MaxSF-']

iterables = [A, B]
index = pd.MultiIndex.from_product(iterables)

C = []

for i in [maxBMs, maxSFs_plus, maxSFs_minus]:
    C.extend(i)

# for i in range(len(loads)):
#     for j in [maxBMs, maxSFs_plus, maxSFs_minus]:
#         C.append(j[i])

df = pd.DataFrame(C, index=index, columns=[span / 8 * i for i in range(9)])
# print(df.loc[('ClassA', 'MaxSF-')])   ## you can navigate using loc, iloc

new_row = df.loc['MaxSF+'].where(df.loc['MaxSF+'] > abs(df.loc['MaxSF-']), abs(df.loc['MaxSF-']))

new_row.index = pd.MultiIndex.from_product([['MaxSF'], B])
df = pd.concat([df, new_row])

# df.to_excel('outputs/loads.xlsx') # uncomment to save as excel file

how to read Excel file

In [5]:
# df = pd.read_excel('outputs/loads.xlsx', index_col=[0, 1])
# ###get index names###
#
# A = df.index.get_level_values(0).drop_duplicates().to_list()
# B = df.index.get_level_values(1).drop_duplicates().to_list()
# print(A, B)

impact factor

In [6]:
IF = [impact(i.name, span) for i in vehicles]
IF

[0.088, 0.088, 0.088]

combination for given carriageway

In [7]:
carriageway = Carriageway(width=15)
combinations = carriageway.combinations()
combination_val = [i.get_value() for i in combinations]

eccentricity calculation

In [8]:
list_combinations = []
list_max_e = []
list_plot = []
for i in combinations:
    list_combinations.append([i.classA, i.class70Rw, i.class70Rt])
    arrangements = i.arrangements()
    list_arrangements = []
    eccentricity = []
    max_e = 0
    for k in arrangements:
        if k.check_exceedance() * k.check_from_right() == 1:
            list_arrangements.append(k.veh)
            e = k.eccentricity()
            eccentricity.append(e)
            max_e = e if abs(e) > abs(max_e) else max_e
    list_max_e.append(max_e)

list_max_e

[-0.950000000000001,
 -3.048814655172414,
 -3.0073275862068964,
 -1.8800000000000003,
 -1.8525000000000003,
 -1.825]

max reactions for given combination

In [9]:
maxBMs = [(df.loc['MaxBM'][25]*IF).dot(combination_val[i]) for i in range(len(combinations))]
maxSFs = [(df.loc['MaxSF'][25]*IF).dot(combination_val[i]) for i in range(len(combinations))]
combination_val, maxBMs, maxSFs

([[4, 0, 0], [2, 1, 0], [2, 0, 1], [0, 2, 0], [0, 1, 1], [0, 0, 2]],
 [1945.2047999999998,
  1929.8136,
  1706.7027999999998,
  1914.4224,
  1691.3116,
  1468.2007999999998],
 [61.657024, 65.810624, 58.776432, 69.964224, 62.930032, 55.89583999999999])

In [10]:
df

0.00      6.25      12.50      18.75      25.00  \
MaxBM  ClassA       0.000  2361.037  4068.075   5120.737   5526.150   
       Class70RW    0.000  4826.025  8200.950  10182.475  10877.400   
       Class70RT    0.000  3651.438  6259.675   7823.212   8342.050   
MaxSF+ ClassA     445.318   376.622   307.926    238.122    169.426   
       Class70RW  895.524   771.524   647.524    521.524    397.524   
       Class70RT  667.590   579.390   492.590    404.390    317.590   
MaxSF- ClassA       0.000   -12.030   -51.234   -106.602   -175.162   
       Class70RW    0.000   -17.024   -87.536   -208.476   -332.476   
       Class70RT    0.000   -54.810  -143.010   -229.810   -316.610   
MaxSF  ClassA     445.318   376.622   307.926    238.122    175.162   
       Class70RW  895.524   771.524   647.524    521.524    397.524   
       Class70RT  667.590   579.390   492.590    404.390    317.590   

                      31.25     37.50     43.75    50.00  
MaxBM  ClassA      5340.038  4327.675  2494.638    0.000  
       Class70RW  10163.225  8053.750  4611.975    0.000  
       Class70RT   7823.588  6260.425  3652.563    0.000  
MaxSF+ ClassA        99.676    43.520    13.056    0.000  
       Class70RW    271.524   148.964    44.948    0.000  
       Class70RT    229.390   142.590    54.390    0.000  
MaxSF- ClassA      -244.966  -313.662  -383.466 -452.162  
       Class70RW   -458.476  -582.476  -708.476 -832.476  
       Class70RT   -404.810  -491.610  -579.810 -666.610  
MaxSF  ClassA       244.966   313.662   383.466  452.162  
       Class70RW    458.476   582.476   708.476  832.476  
       Class70RT    404.810   491.610   579.810  666.610

make a dataframe for corresponding eccentricity + combinations

In [ ]:
df1 = pd.DataFrame(list_combinations, columns=['ClassA', 'Class70Rw', 'Class70Rt'],
                  index=[f'comb{i + 1}' for i in range(len(list_combinations))])
df1['MaxEccentricity'] = list_max_e

export eccentricity as excel file (uncomment)

In [13]:
# df1.to_excel('outputs/max_e.xlsx')

In [15]:
df1

,ClassA,Class70Rw,Class70Rt,MaxEccentricity
comb1,4,0,0,-0.950000
comb2,2,1,0,-3.048815
comb3,2,0,1,-3.007328
comb4,0,2,0,-1.880000
comb5,0,1,1,-1.852500
comb6,0,0,2,-1.825000
